In [1]:
%matplotlib ipympl
def figure(name, nrows=1, ncols=1, *args, **kwargs):
    plt.close(name)
    return plt.subplots(nrows, ncols, num=name, *args, **kwargs)

import numpy as np
import pylab as plt
plt.style.use('default')

In [13]:
from scipy.integrate import solve_ivp
from ipywidgets import HBox, IntSlider, FloatSlider
from numpy import sin, cos

In [103]:
deg2rad = lambda deg: np.pi * deg / 180
# m1, m2, M
def get_cart_coords(y, t):
    z = l * cos(y[0])
    cart = np.zeros_like((9, y.shape[1]))
    cart[0] = z * sin(w * t)
    cart[1] = z * cos(w * t)
    cart[2] = z
    cart[3] = -z * sin(w * t)
    cart[4] = -z * cos(w * t)
    cart[5] = z
    # Mz
    cart[8] = 2 * z
    return cart

def energy(y):
    T = 2*M*l**2*y[1]**2*sin(y[0])**2 + m*l**2*y[1]**2 + m*l**2*w**2*sin(y[0])**2
    U = -2*(m+M)*g*l*cos(y[0]) 
    return np.array((U, T))

In [154]:
# phi, dot(phi)
def derivative(t, y):
    dy = np.zeros(2)
    dy[0] = y[1]
    dy[1] = ((m*w**2 - 2*M*y[1]**2)*sin(y[0])*cos(y[0]) - g/l*(m+M)*sin(y[0]))/(2*M*sin(y[0])**2 + m)
#     dy[1] -= 0.2 * y[1]
    return dy

In [155]:
times = np.linspace(0, 60, 1000)
m = 1
M = 2
g = 9.81
l = 1
w = 5.5

Y0 = [deg2rad(1), 0.]
# Y0 = [np.pi*0.95, 0.]
print(np.sqrt(g/l * (m+M)/m))

5.424942396007538


In [156]:
sol = solve_ivp(derivative, [0, 100], Y0, t_eval=times, 
                method='DOP853',
#                 method='LSODA',
#                 method='RK23', 
                rtol=1e-8,
                atol=1e-10
               )
# coords = get_cart_coords(sol.y)
fig, ax = figure('example', 1, 2, figsize=(12, 4))

ax[0].plot(sol.t, sol.y[0]/np.pi, label='phi')
# ax[0].plot(sol.t, sol.y[1], label='p')
steady = g/(l*w**2) * (m+M)/m
if steady <= 1:
    ax[0].plot(sol.t, [np.arccos(steady)/np.pi]*len(times), 'k-')
ax[1].plot(sol.t, energy(sol.y).T)
ax[0].legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [78]:
sol.message

'The solver successfully reached the end of the integration interval.'